# Create json files for each experiment

Load a list of experiments by `session_id` and create the necessary json files for it in the `./data/{session_id}` folder.

In [25]:
import pandas as pd
import json
from ast import literal_eval
import os
import db_queries

In [19]:
# load in the data
session_id = 'S0001' # db_builder.get_next_session_id()
result = db_queries.get_complete_session(session_id)
df = pd.DataFrame(columns=['session_id', 'experiment_id', 'container_id', 'objects'], data=result )

# evaluate objects to list
df['objects'] = df['objects'].apply(lambda x: x.split(','))

In [21]:
def build_json(session_id, experiment_id, container_id, objects):
    return {
        'session_id': session_id.strip(),
        'experiment_id': experiment_id.strip(),
        'container_id': container_id.strip(),
        'status': 'pending',
        'n_objects': len(objects),
        'aligned': False,
        'annotations': [{'object': item.strip()} for item in objects]
    }

In [22]:
experiments = []

for i, row in df.iterrows():
    json = build_json(row['session_id'], row['experiment_id'], row['container_id'], row['objects'])
    experiments.append(json)

In [26]:
# create folder
os.mkdir(f'./data/{session_id}')

In [27]:
# save files
for experiment in experiments:
    with open(f'./data/{session_id}/{experiment["experiment_id"]}.json', 'bw') as f:
        # save nicely formatted json
        f.write(json.dumps(experiment, indent=4).encode('utf-8'))